# Basic Example: Cache with LRU Replacement

This example demonstrates a cache model with LRU (Least Recently Used) replacement:
- **Cache**: Stores m items from a universe of n items
- **Replacement**: LRU eviction policy
- **Access pattern**: Uniform distribution over items
- **Closed model**: Single job circulating between delay and cache

The model tracks hit and miss rates for the cache.

In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
def cache_replc_lru(n=5, m=2):
    """
    Create a cache model with LRU replacement.
    
    Args:
        n: Number of items in the universe
        m: Cache capacity (number of items that can be stored)
    """
    model = Network('model')
    
    # Block 1: nodes
    delay = Delay(model, 'Delay')
    cache_node = Cache(model, 'Cache', n, m, ReplacementStrategy.LRU)
    
    # Block 2: classes
    job_class = ClosedClass(model, 'JobClass', 1, delay, 0)
    hit_class = ClosedClass(model, 'HitClass', 0, delay, 0)
    miss_class = ClosedClass(model, 'MissClass', 0, delay, 0)
    
    # Set service time at delay node
    delay.set_service(job_class, Exp(1))
    
    # Set cache access pattern (uniform over items)
    p_access = DiscreteSampler([1/n] * n)
    cache_node.set_read(job_class, p_access)
    
    # Set hit and miss classes
    cache_node.set_hit_class(job_class, hit_class)
    cache_node.set_miss_class(job_class, miss_class)
    
    # Block 3: topology
    P = model.init_routing_matrix()
    P.add_route(job_class, delay, cache_node, 1.0)
    P.add_class_switch(hit_class, job_class, cache_node, delay, 1.0)
    P.add_class_switch(miss_class, job_class, cache_node, delay, 1.0)
    model.link(P)
    
    return model

# Create the model
model = cache_replc_lru(n=5, m=2)

## About LRU Caching

**LRU (Least Recently Used)** replacement:
- When the cache is full and a miss occurs, evict the least recently accessed item
- Exploits temporal locality in access patterns
- Commonly used in CPU caches, web caches, databases

**Model parameters**:
- n=5 items in universe
- m=2 cache capacity
- Uniform access probability: each item has 1/5 = 0.2 chance of being accessed

With uniform access and LRU, the expected hit rate depends on the cache-to-item ratio.

In [3]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, keep=False)
avg_node_table_ctmc = solver_ctmc.avg_node_table()
print("\nCTMC Solver:")
print(avg_node_table_ctmc)

# MVA Solver
model.reset()
solver_mva = MVA(model)
avg_node_table_mva = solver_mva.avg_node_table()
print("\nMVA Solver:")
print(avg_node_table_mva)


=== Solver Results ===
Station JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
  Delay JobClass   1.0   1.0    1.0     1.0   1.0   1.0

CTMC Solver:
  Station  JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0   Delay  JobClass   1.0   1.0    1.0     1.0   1.0   1.0
Station JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
  Delay JobClass   1.0   1.0    1.0     1.0   1.0   1.0

MVA Solver:
  Station  JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0   Delay  JobClass   1.0   1.0    1.0     1.0   1.0   1.0


/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:271: RuntimeWarning: divide by zero encountered in divide
  deltaclass = (Nchain - 1) / Nchain
/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:119: RuntimeWarning: divide by zero encountered in divide
  deltaclass = (Nchain_in - 1) / Nchain_in
